In [1]:
import numpy as np
import pandas as pd

from transformers import pipeline, AutoTokenizer
from tqdm import tqdm

In [3]:
DATA_PATH = f"D:\code\DACON-Korean-sentence-relation-classification\open\open"
#train = pd.read_csv(os.path.join(DATA_PATH, "train_data.csv"))
#test = pd.read_csv(os.path.join(DATA_PATH, "test_data.csv") )
#submission = pd.read_csv(os.path.join(DATA_PATH, "sample_submission.csv"))

In [7]:
max_len = 70
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

device = torch.device("cuda:0")

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Huffon/klue-roberta-base-nli")
model = pipeline("text-classification", model="Huffon/klue-roberta-base-nli", return_all_scores=False)

Downloading: 100%|██████████| 926/926 [00:00<00:00, 307kB/s]
Downloading: 100%|██████████| 243k/243k [00:00<00:00, 323kB/s]
Downloading: 100%|██████████| 483k/483k [00:00<00:00, 634kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 109kB/s]
Downloading: 100%|██████████| 422M/422M [00:23<00:00, 18.8MB/s]


In [4]:
df_test = pd.read_csv(os.path.join(DATA_PATH, "test_data.csv") )
X_test = df_test["premise"] + " " + tokenizer.sep_token + " " + df_test["hypothesis"]

In [5]:
y_preds = []
for i in tqdm(range(X_test.shape[0])):
    y_pred = model(X_test[i])[0]["label"].lower()
    y_preds.append(y_pred)

y_preds = np.array(y_preds)

100%|██████████| 1666/1666 [01:04<00:00, 25.73it/s]


In [8]:
df_test["label"] = y_preds
df_submission = df_test.loc[:, ["index", "label"]]
df_submission.to_csv("./results/Huffon-klue-roberta-base-nli.csv", index=False)